# Import and Install Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#download and install langchain dependencies
!pip install -qU qdrant-client langchain-openai langchain-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 30.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.


In [2]:
# importing all libraries needed
import pandas as pd #for data processing
import getpass, os #for input password interface

from langchain_openai import OpenAIEmbeddings, ChatOpenAI #for embedding and LLM service
from uuid import uuid4 #for generate id with uuid format

from langchain_core.documents import Document #for document format that stored to vector database collection
from langchain_qdrant import QdrantVectorStore #for langchain - qdrant vector database connector
from qdrant_client import QdrantClient #for qdrant client set up
from qdrant_client.http.models import Distance, VectorParams #for distance method and configuration of vector parameters

# Setup Environment API_KEY

In [3]:
#input openai api key
from google.colab import userdata

if userdata.get('OPENAI_KEY'):
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')
else:
  os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_KEY")

print('OPENAI_API_KEY telah diinput')

OPENAI_API_KEY telah diinput


In [4]:
#input qdrant api key
if userdata.get('QDRANT_KEY'):
  api_key_qdrant = userdata.get('QDRANT_KEY')
else:
  api_key_qdrant = getpass.getpass("QDRANT_KEY")

print('QDRANT_API_KEY telah diinput')

QDRANT_API_KEY telah diinput


In [5]:
#input qdrant api key
if userdata.get('QDRANT_URL'):
  url_qdrant = userdata.get('QDRANT_URL')
else:
  url_qdrant = getpass.getpass("Input URL QDRANT CLOUD")

print('QDRANT_URL telah diinput')

QDRANT_URL telah diinput


In [6]:
url_qdrant

'https://56199c0a-342b-4a06-8483-e5857786ab3b.us-east4-0.gcp.cloud.qdrant.io'

# Define LLM and Processing Dataset

In [7]:
#define embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#define llm model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [10]:
#define directory path of dataset
data_path = "/content/imdb_top_1000.csv"

In [11]:
#read dataset with pandas
df = pd.read_csv(data_path)
print(df.shape) #show total row and column with format (row, column)
df.head() #show first 5 record of dataset

(1000, 16)


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


In [14]:
data = df.copy() #copy data table
# data = data.iloc[:1000,].reset_index(drop=True)
data = data.dropna(subset=['Certificate', 'Meta_score', 'Gross']).reset_index(drop=True) #delete rows that contain missing values
data.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    714 non-null    object 
 1   Series_Title   714 non-null    object 
 2   Released_Year  714 non-null    object 
 3   Certificate    714 non-null    object 
 4   Runtime        714 non-null    object 
 5   Genre          714 non-null    object 
 6   IMDB_Rating    714 non-null    float64
 7   Overview       714 non-null    object 
 8   Meta_score     714 non-null    float64
 9   Director       714 non-null    object 
 10  Star1          714 non-null    object 
 11  Star2          714 non-null    object 
 12  Star3          714 non-null    object 
 13  Star4          714 non-null    object 
 14  No_of_Votes    714 non-null    int64  
 15  Gross          714 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 89.4+ KB


In [16]:
#looping and set the data into Document format
documents = []
#setup unique id
uuids = [str(uuid4()) for _ in range(data.shape[0])]

for i in range(data.shape[0]):
  Title = data["Series_Title"][i]
  Released_Year = data["Released_Year"][i]
  Certificate = data["Certificate"][i]
  Duration = data["Runtime"][i]
  Genre = data["Genre"][i]
  IMDB_Rating = data["IMDB_Rating"][i]
  Overview = data["Overview"][i]
  Meta_Score  = data["Meta_score"][i]
  Director  = data["Director"][i]
  Star1 = data["Star1"][i]
  Star2 = data["Star2"][i]
  Star3 = data["Star3"][i]
  Star4 = data["Star4"][i]
  No_of_Votes = data["No_of_Votes"][i]
  Gross = data["Gross"][i]
  doc = Document(
      page_content = (
          f"Title: {Title} ({Released_Year}). "
          f"Genre: {Genre}. "
          f"Director: {Director}. "
          f"Starring: {Star1}, {Star2}, {Star3}, and {Star4}. "
          f"This film has a score of {IMDB_Rating} and is {Duration} long."
      ),
      metadata = {
          "movie_id": uuids[i], # Use the generated UUID as the unique ID
          "title": str(Title),
          "released_year": str(Released_Year),
          "certificate": str(Certificate),
          "duration": str(Duration),
          "genre": str(Genre),
          "rating": str(IMDB_Rating),
          "overview": str(Overview),
          "meta_score": str(Meta_Score),
          "director": str(Director),
          "star1": str(Star1),
          "star2": str(Star2),
          "star3": str(Star3),
          "star4": str(Star4),
          "no_of_votes": str(No_of_Votes),
          "gross": str(Gross),
      },
  )
  documents.append(doc)

In [17]:
#show a sample document
documents[0]

Document(metadata={'movie_id': 'cfe184bf-cc22-483c-8579-9b760a1a6b25', 'title': 'The Shawshank Redemption', 'released_year': '1994', 'certificate': 'A', 'duration': '142 min', 'genre': 'Drama', 'rating': '9.3', 'overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'meta_score': '80.0', 'director': 'Frank Darabont', 'star1': 'Tim Robbins', 'star2': 'Morgan Freeman', 'star3': 'Bob Gunton', 'star4': 'William Sadler', 'no_of_votes': '2343110', 'gross': '28,341,469'}, page_content='Title: The Shawshank Redemption (1994). Genre: Drama. Director: Frank Darabont. Starring: Tim Robbins, Morgan Freeman, Bob Gunton, and William Sadler. This film has a score of 9.3 and is 142 min long.')

# Vector Database Process

In [18]:
#save document to qdrant
qdrant = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    url=url_qdrant,
    prefer_grpc=True,
    api_key=api_key_qdrant,
    collection_name="product_documents",
)

In [ ]:
# # for deleting collection

# client = QdrantClient(
#   url= url_qdrant,
#   api_key = api_key_qdrant
# )
# client.delete_collection(collection_name="product_documents")

In [19]:
#setup qdrant client
client = QdrantClient(
  url= url_qdrant,
  api_key = api_key_qdrant
)
#get all collection in your qdrant vector database
collections_response = client.get_collections()
print("Collections:", collections_response.collections)

Collections: [CollectionDescription(name='product_documents')]


# Query in VectorDB

In [20]:
#qdrant vectorstore with specific collection name
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="product_documents",
    url=url_qdrant,
    api_key=api_key_qdrant
)

In [22]:
#retrieve data from qdrant
results = qdrant.similarity_search(
    "commedy movie",
    k=2
)
results

[Document(metadata={'gross': '2,500,000', 'certificate': 'U', 'title': 'The King of Comedy', 'released_year': '1982', 'star1': 'Robert De Niro', 'director': 'Martin Scorsese', 'no_of_votes': '88511', 'duration': '109 min', 'overview': 'Rupert Pupkin is a passionate yet unsuccessful comic who craves nothing more than to be in the spotlight and to achieve this, he stalks and kidnaps his idol to take the spotlight for himself.', 'star3': 'Diahnne Abbott', 'star2': 'Jerry Lewis', 'rating': '7.8', 'star4': 'Sandra Bernhard', 'meta_score': '73.0', 'movie_id': '6b325490-ffc4-4a21-9bb8-9a30544ccf2b', 'genre': 'Comedy, Crime, Drama', '_id': '86d41070-366c-40fc-a9e0-81646ee9b5dd', '_collection_name': 'product_documents'}, page_content='Title: The King of Comedy (1982). Genre: Comedy, Crime, Drama. Director: Martin Scorsese. Starring: Robert De Niro, Jerry Lewis, Diahnne Abbott, and Sandra Bernhard. This film has a score of 7.8 and is 109 min long.'),
 Document(metadata={'star3': 'Diahnne Abbott'

In [23]:
#parsing page content and metadata from retrieved data
for res in results:
    print(f"Result Page Content:  {res.page_content}\nResult Metadata: [{res.metadata}]")
    print("-----"*20)

Result Page Content:  Title: The King of Comedy (1982). Genre: Comedy, Crime, Drama. Director: Martin Scorsese. Starring: Robert De Niro, Jerry Lewis, Diahnne Abbott, and Sandra Bernhard. This film has a score of 7.8 and is 109 min long.
Result Metadata: [{'gross': '2,500,000', 'certificate': 'U', 'title': 'The King of Comedy', 'released_year': '1982', 'star1': 'Robert De Niro', 'director': 'Martin Scorsese', 'no_of_votes': '88511', 'duration': '109 min', 'overview': 'Rupert Pupkin is a passionate yet unsuccessful comic who craves nothing more than to be in the spotlight and to achieve this, he stalks and kidnaps his idol to take the spotlight for himself.', 'star3': 'Diahnne Abbott', 'star2': 'Jerry Lewis', 'rating': '7.8', 'star4': 'Sandra Bernhard', 'meta_score': '73.0', 'movie_id': '6b325490-ffc4-4a21-9bb8-9a30544ccf2b', 'genre': 'Comedy, Crime, Drama', '_id': '86d41070-366c-40fc-a9e0-81646ee9b5dd', '_collection_name': 'product_documents'}]
-----------------------------------------

In [24]:
#try to retrieve data from vector database with similarity_search_with_score
#this method will give you additional information about similarity score
results = qdrant.similarity_search_with_score(
    "action movie",
    k=2
)
results

[(Document(metadata={'no_of_votes': '329592', 'star4': 'Radha Mitchell', 'duration': '146 min', 'movie_id': '18a88be7-5817-48ba-8f20-ce1d264a4d8f', 'genre': 'Action, Crime, Drama', 'star1': 'Denzel Washington', 'director': 'Tony Scott', 'certificate': 'UA', 'meta_score': '47.0', 'overview': 'In Mexico City, a former CIA operative swears vengeance on those who committed an unspeakable act against the family he was hired to protect.', 'star2': 'Christopher Walken', 'gross': '77,911,774', 'title': 'Man on Fire', 'star3': 'Dakota Fanning', 'rating': '7.7', 'released_year': '2004', '_id': 'b6514c38-9b37-42a2-b30a-5ab5424ef6d4', '_collection_name': 'product_documents'}, page_content='Title: Man on Fire (2004). Genre: Action, Crime, Drama. Director: Tony Scott. Starring: Denzel Washington, Christopher Walken, Dakota Fanning, and Radha Mitchell. This film has a score of 7.7 and is 146 min long.'),
  0.43246907),
 (Document(metadata={'star4': 'Radha Mitchell', 'duration': '146 min', 'star1': 'D

In [25]:
#parsing page content, metadata, and similarity score for each retrieved document
for res,score in results:
    print(f"Score: {score}\nResult Page Content:  {res.page_content}\nResult Metadata: [{res.metadata}]")
    print("-----"*20)

Score: 0.43246907
Result Page Content:  Title: Man on Fire (2004). Genre: Action, Crime, Drama. Director: Tony Scott. Starring: Denzel Washington, Christopher Walken, Dakota Fanning, and Radha Mitchell. This film has a score of 7.7 and is 146 min long.
Result Metadata: [{'no_of_votes': '329592', 'star4': 'Radha Mitchell', 'duration': '146 min', 'movie_id': '18a88be7-5817-48ba-8f20-ce1d264a4d8f', 'genre': 'Action, Crime, Drama', 'star1': 'Denzel Washington', 'director': 'Tony Scott', 'certificate': 'UA', 'meta_score': '47.0', 'overview': 'In Mexico City, a former CIA operative swears vengeance on those who committed an unspeakable act against the family he was hired to protect.', 'star2': 'Christopher Walken', 'gross': '77,911,774', 'title': 'Man on Fire', 'star3': 'Dakota Fanning', 'rating': '7.7', 'released_year': '2004', '_id': 'b6514c38-9b37-42a2-b30a-5ab5424ef6d4', '_collection_name': 'product_documents'}]
----------------------------------------------------------------------------

In [26]:
#retrieve data with non-default search_type.
#basically, default search_type is similarity_score_threshold, the code below shows how to retrieve with another search_type
retriever = qdrant.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("action movie")

[Document(metadata={'no_of_votes': '329592', 'star4': 'Radha Mitchell', 'duration': '146 min', 'movie_id': '18a88be7-5817-48ba-8f20-ce1d264a4d8f', 'genre': 'Action, Crime, Drama', 'star1': 'Denzel Washington', 'director': 'Tony Scott', 'certificate': 'UA', 'meta_score': '47.0', 'overview': 'In Mexico City, a former CIA operative swears vengeance on those who committed an unspeakable act against the family he was hired to protect.', 'star2': 'Christopher Walken', 'gross': '77,911,774', 'title': 'Man on Fire', 'star3': 'Dakota Fanning', 'rating': '7.7', 'released_year': '2004', '_id': 'b6514c38-9b37-42a2-b30a-5ab5424ef6d4', '_collection_name': 'product_documents'}, page_content='Title: Man on Fire (2004). Genre: Action, Crime, Drama. Director: Tony Scott. Starring: Denzel Washington, Christopher Walken, Dakota Fanning, and Radha Mitchell. This film has a score of 7.7 and is 146 min long.')]